<a href="https://colab.research.google.com/github/skj092/Sefty_Helmet_Detection/blob/main/Head_Hardhat_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 

os.chdir("/content/drive/MyDrive/Projects/Head-Hardhet-Detection")

In [35]:
# # # Unzip dataset
# !unzip -q HardHat_Dataset/Dataset.zip  -d HardHat_Dataset/

In [48]:
!unzip -q HardHat_Dataset/Dataset/HardHat_Dataset.zip -d HardHat_Dataset

In [49]:
# this script will convert xml to yolo format

import os
from glob import glob
from pathlib import Path
import cv2
import xml.etree.ElementTree as ET
import numpy as np 
from tqdm import tqdm


root = Path('HardHat_Dataset')
annotations = list(root.glob('annotations/*.xml'))
output_path = root / 'labels'

# create a directory for yolo format
if not os.path.exists(root / 'labels'):
    os.makedirs(root / 'labels')

lbl = ['head', 'helmet','person']

for annot in tqdm(annotations):
    tree = ET.parse(annot)
    root = tree.getroot()
    img_path = os.path.join('HardHat_Dataset/images', os.path.basename(annot).replace('.xml', '.png'))
    img = cv2.imread(img_path)
    height, width, _ = img.shape
    objects = root.findall('object')
    yolo_data = []
    for obj in objects:
        name = obj.find('name').text
        # replace the class "person" with "head"
        if name == 'person':
            name = 'head'
        xmin = int(obj.find('bndbox/xmin').text)
        ymin = int(obj.find('bndbox/ymin').text)
        xmax = int(obj.find('bndbox/xmax').text)
        ymax = int(obj.find('bndbox/ymax').text)
        x_center = (xmin + xmax) / (2 * width)
        y_center = (ymin + ymax) / (2 * height)
        w = (xmax - xmin) / width
        h = (ymax - ymin) / height
        yolo_data.append([lbl.index(name), x_center, y_center, w, h])
    yolo_data = np.array(yolo_data)
    np.savetxt(output_path / (os.path.basename(annot).replace('.xml', '.txt')), yolo_data, fmt=['%d', '%f', '%f', '%f', '%f'])

100%|██████████| 4750/4750 [01:33<00:00, 50.68it/s]


In [50]:
# Dividing the data into training and testing sets

# Importing the libraries
import os 
from glob import glob
import random 
import cv2 
import shutil
from pathlib import Path
from tqdm import tqdm

p = Path('.')
data_dir = Path('HardHat_Dataset')
img_dir = data_dir/'images/'
annot_dir = data_dir/'labels'

# Creating a list of all the images
all_images = list(p.glob(f'{img_dir}/*.png'))

# shuffling the data
random.shuffle(all_images)
# Splitting the data into training and testing sets
train_images = all_images[:int(0.8*len(all_images))]
test_images = all_images[int(0.8*len(all_images)):]
# print('langth of train and test set are:', len(train_images), len(test_images))

# # Splitting the data 
if not os.path.exists(data_dir/'processed'):
    os.makedirs(data_dir/'processed')
    os.makedirs(data_dir/'processed'/'train')
    os.makedirs(data_dir/'processed'/'test')
    os.makedirs(data_dir/'processed'/'train'/'images')
    os.makedirs(data_dir/'processed'/'train'/'labels')
    os.makedirs(data_dir/'processed'/'test'/'images')
    os.makedirs(data_dir/'processed'/'test'/'labels')


# removing train and test folder 
# shutil.rmtree(path=data_dir/'processed')


# Copying the images and annotations into the train and test folder
for source in tqdm(all_images):
    destination_img = os.path.join(data_dir/'processed'/'train'/'images', os.path.basename(source))
    destination_annot = os.path.join(data_dir/'processed'/'train'/'labels', os.path.basename(source).replace('.png', '.txt'))
    shutil.copy(source, destination_img)
    source_annot = os.path.join(annot_dir, os.path.basename(source).replace('.png', '.txt'))
    shutil.copy(source_annot, destination_annot)

for source in tqdm(test_images):
    destination_img = os.path.join(data_dir/'processed'/'test'/'images', os.path.basename(source))
    destination_annot = os.path.join(data_dir/'processed'/'test'/'labels', os.path.basename(source).replace('.png', '.txt'))
    shutil.copy(source, destination_img)
    source_annot = os.path.join(annot_dir, os.path.basename(source).replace('.png', '.txt'))
    shutil.copy(source_annot, destination_annot)

100%|██████████| 950/950 [00:15<00:00, 63.16it/s]


In [51]:
!ls

Dataset  data.yaml  HardHat_Dataset  Images  label_fix.py  yolov5


In [52]:
yaml_text = """train: ../HardHat_Dataset/processed/train
val: ../HardHat_Dataset/processed/test

nc: 2
names: ['Head','Helmet']"""

In [53]:
with open("data.yaml", 'w') as f:
    f.write(yaml_text)
%cat data.yaml

train: ../HardHat_Dataset/processed/train
val: ../HardHat_Dataset/processed/test

nc: 2
names: ['Head','Helmet']

In [54]:
!pwd

/content/drive/MyDrive/Projects/Head-Hardhet-Detection


In [55]:
!ls

Dataset  data.yaml  HardHat_Dataset  Images  label_fix.py  yolov5


In [56]:
# check the assigned GPU type
!nvidia-smi

Wed Nov  9 02:50:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [57]:
# YOLO 
# !git clone https://github.com/ultralytics/yolov5  && cd yolov5 && pip install -r requirements.txt

In [58]:
!python ./yolov5/train.py --img 512 --batch 16 --epochs 100 --workers 2 --data data.yaml --cfg "./yolov5/models/yolov5s.yaml" --name yolov5x_helmet --cache

train: weights=yolov5/yolov5s.pt, cfg=./yolov5/models/yolov5s.yaml, data=data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=yolov5/runs/train, name=yolov5x_helmet, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 3 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.2-228-g6ae3dff Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.

In [64]:
# Validation

!python ./yolov5/val.py --img 350 --batch-size 16 --data data.yaml --weights "yolov5/runs/train/yolov5x_helmet5/weights/best.pt" --conf 0.1

val: data=data.yaml, weights=['yolov5/runs/train/yolov5x_helmet5/weights/best.pt'], batch_size=16, imgsz=350, conf_thres=0.1, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
WARNING ⚠️ confidence threshold 0.1 > 0.001 produces invalid results
YOLOv5 🚀 v6.2-228-g6ae3dff Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ --img-size 350 must be multiple of max stride 32, updating to 352
val: Scanning '/content/drive/MyDrive/Projects/Head-Hardhet-Detection/HardHat_Dataset/processed/test/labels.cache' images and labels... 950 found, 0 missing, 0 empty, 0 corrupt: 100% 950/950 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 1

# Prediction on Test Images

In [62]:
#unzipping test images

!unzip -q HardHat_Dataset/Dataset/HardHat_Test_Images.zip -d testimages

In [ ]:
# Detection on test dataset
!python ./yolov5/detect.py --weights "yolov5/runs/train/yolov5x_helmet5/weights/best.pt" --img 512 --conf 0.1 --source testimages/Images --save-txt --save-conf --exist-ok


# Prediction on YouTube Video

In [67]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 2.9 MB/s 


In [68]:
from pytube import YouTube
YouTube('https://www.youtube.com/watch?v=6PoPwZ0WO9w').streams.first().download()
yt = YouTube('https://www.youtube.com/watch?v=6PoPwZ0WO9w')

In [69]:
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download()

'/content/drive/MyDrive/Projects/Head-Hardhet-Detection/Top 10 Safety Vest For Construction For Men And Women.mp4'

In [70]:
!mv "Top 10 Safety Vest For Construction For Men And Women.mp4" test.mp4

In [ ]:
# extract 10 minutes worth of video frames of it
!yes | ffmpeg -ss 00:00:05 -i test.mp4 -t 00:10:50 -vf fps=30 out.mp4

In [ ]:
!python ./yolov5/detect.py --source out.mp4 --conf 0.1 --weights "yolov5/runs/train/yolov5x_helmet5/weights/best.pt" 

In [73]:
!ffmpeg -i yolov5/runs/detect/exp2/out.mp4 -vf fps=30 -vcodec libx264 output.mp4

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

In [74]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

In [75]:
HTML("""

      %s" type="video/mp4">

""" % data_url)